# Import Lib & Data


In [ ]:
import numpy as np
import pandas as pd
import scipy.io as sio
import tensorly as tl
from tensorly.decomposition import parafac
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score

In [1]:
import scipy.io as sio
import pandas as pd

df_mat = sio.loadmat("Acar_data/EEM_NMR_LCMS.mat")
df_cts = pd.read_csv("Acar_data/concentrations.txt", delim_whitespace=True)

/var/folders/55/brtsb88x4wz9n5kxdtldl3940000gn/T/ipykernel_90189/452374942.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_cts = pd.read_csv("Acar_data/concentrations.txt", delim_whitespace=True)


In [2]:
df_cts.head(3)

,Val-Tyr-Val,Trp-Gly,Phe,Malto,Propanol
1,5.0,0.0,0.0,0.0,0.0
2,0.0,5.0,0.0,0.0,0.0
3,0.0,0.0,5.0,0.0,0.0


In [3]:
print(df_mat.keys())
print(df_mat["X"].dtype.names)

dict_keys(['__header__', '__version__', '__globals__', 'Y', 'X', 'Z'])
('name', 'type', 'author', 'date', 'moddate', 'imagesize', 'imagemode', 'data', 'label', 'axisscale', 'imageaxisscale', 'title', 'class', 'include', 'classlookup', 'axistype', 'imageaxistype', 'description', 'userdata', 'datasetversion', 'history', 'uniqueid')


In [ ]:
data_dic = {}
dimensions = ["X", "Y", "Z"]
for dim in dimensions:
    mesurement_technique = df_mat[dim]["name"][0][0][0]
    data = df_mat[dim]["data"][0][0]
    data_dic[mesurement_technique] = data
    print(f"Extracted {mesurement_technique} data with shape {data.shape}")

X_eem = data_dic["EEM"]
X_nmr = data_dic["3-way NMR"]
X_lcms = data_dic["LCMS"]

Extracted EEM data with shape (28, 251, 21)
Extracted 3-way NMR data with shape (28, 13324, 8)
Extracted LCMS data with shape (28, 168)


# Check for missing values


In [ ]:
print("NaNs ?", np.isnan(X_lcms).any())
print("Inf ?", np.isinf(X_lcms).any())
nb_of_values = X_lcms.shape[0] * X_lcms.shape[1]
nb_nan = np.isnan(X_lcms).sum()
print(f"NaN ratio : {nb_nan} / {nb_of_values} = {nb_nan/nb_of_values:.2%}")

NaNs ? False
Inf ? False
NaN ratio : 0 / 4704 = 0.00%


In [ ]:
print("NaNs ?", np.isnan(X_nmr).any())
print("Inf ?", np.isinf(X_nmr).any())
nb_of_values = X_nmr.shape[0] * X_nmr.shape[1] * X_nmr.shape[2]
nb_nan = np.isnan(X_nmr).sum()
print(f"NaN ratio : {nb_nan} / {nb_of_values} = {nb_nan/nb_of_values:.2%}")

NaNs ? False
Inf ? False
NaN ratio : 0 / 2984576 = 0.00%


In [ ]:
print("NaNs ?", np.isnan(X_eem).any())
print("Inf ?", np.isinf(X_eem).any())
nb_of_values = X_eem.shape[0] * X_eem.shape[1] * X_eem.shape[2]
nb_nan = np.isnan(X_eem).sum()
print(f"NaN ratio : {nb_nan} / {nb_of_values} = {nb_nan/nb_of_values:.2%}")

NaNs ? True
Inf ? False
NaN ratio : 11508 / 147588 = 7.80%


# PARAFAC


In [ ]:
SEED = 42
np.random.seed(SEED)

CP_RANK_NMR = 3
CP_RANK_EEM = 3
PCA_RANK_LCMS = 5


# Factorisation CP pour NMR et EEM
def cp_scores(tensor, rank):
    """
    Applique PARAFAC à un tenseur 3D et retourne les scores des individus (mode 0).
    """
    print(f"Tenseur shape avant CP: {tensor.shape}")

    # Check que le mode 0 = individus (n = 28)
    if tensor.shape[0] != 28:
        raise ValueError(
            "Le mode 0 n'est pas celui des individus. Tenseur mal orienté."
        )

    # PARAFAC
    # factors = parafac(tensor, rank=rank, init='svd', random_state=SEED)
    factors = parafac(
        tensor, rank=rank, n_iter_max=2000, tol=1e-6, init="svd", random_state=SEED
    )
    for i, f in enumerate(factors):
        print(f"{i}. Shape du facteur {i} : {f[0].shape}")

    return tl.to_numpy(factors[1][0])


print("-" * 20, "Performing CP decomposition", "-" * 20)

scores_nmr = cp_scores(X_nmr, CP_RANK_NMR)
scores_eem = cp_scores(X_eem, CP_RANK_EEM)

#  PCA LCMS
pca_lcms = PCA(n_components=PCA_RANK_LCMS, random_state=SEED)
scores_lcms = pca_lcms.fit_transform(X_lcms)

# Fusion
if scores_nmr.shape[0] != scores_lcms.shape[0]:
    scores_nmr = scores_nmr.T
if scores_eem.shape[0] != scores_lcms.shape[0]:
    scores_eem = scores_eem.T

print("-" * 50)
print(f"Shape scores NMR: {scores_nmr.shape}")
print(f"Shape scores EEM: {scores_eem.shape}")
print(f"Shape scores LCMS: {scores_lcms.shape}")
X_fused = np.hstack([scores_nmr, scores_eem, scores_lcms])
print(f"Shape finale des features fusionnés: {X_fused.shape}")


df_fused = pd.DataFrame(
    data=X_fused,
    columns=[f"NMR_{i}" for i in range(scores_nmr.shape[1])]
    + [f"EEM_{i}" for i in range(scores_eem.shape[1])]
    + [f"LCMS_{i}" for i in range(scores_lcms.shape[1])],
)
df_fused.head(3)

-------------------- Performing CP decomposition --------------------
Tenseur shape avant CP: (28, 13324, 8)
0. Shape du facteur 0 : ()
1. Shape du facteur 1 : (28, 3)
Tenseur shape avant CP: (28, 251, 21)


LinAlgError: SVD did not converge

In [19]:
scores_eem

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]])

In [ ]:
concentrations = pd.read_csv("Acar_data/concentrations.txt", sep=r"\s+", index_col=0)
y = concentrations.values
print(f"Shape des concentrations (y): {y.shape}")

# Standardisation et régression Ridge
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_fused)

alphas = np.logspace(-3, 3, 100)
model = RidgeCV(alphas=alphas, cv=5)

# Validation croisée
scores = cross_val_score(model, X_scaled, y, cv=5, scoring="r2")
print(f"R² moyen en validation croisée : {scores.mean():.4f}")